In [1]:
import sys
import torch
import tntorch as tn
import numpy as np
sys.path.append('..')
import lib

In [2]:
rho, train_X, train_y = lib.read_data('../data/2019_12_29_haar_dim4_0.dat')
dim = rho.shape[0]
n_qubits = np.log2(dim)

tensor_rank = 2**n_qubits
batch_size = 100
lr = 1e-3

sigma = lib.simulator.randomMixedState(rho.shape[0])
sigma_real, sigma_imag = [tn.Tensor(x, ranks_tt=tensor_rank, requires_grad=True) for x in [np.real(sigma), np.imag(sigma)]]

epoches = 10

In [3]:
def trace(tensor):
    if len(tensor.shape) == 2:
        return sum([tensor[i,i] for i in range(tensor.shape[0])])
    if len(tensor.shape) == 3:
        return sum([tensor[:, i,i] for i in range(tensor.shape[1])])

def cholesky(sigma_real, sigma_imag):
    sigma_real, sigma_imag = sigma_real.dot(sigma_real, k=1)+sigma_imag.dot(sigma_imag, k=1), sigma_real.dot(sigma_imag, k=1)-sigma_imag.dot(sigma_real, k=1)
    trace = sum([sigma_real[i,i] for i in range(rho.shape[0])])
    sigma_real, sigma_imag = [x/trace for x in [sigma_real, sigma_imag]]
    return sigma_real, sigma_imag
    
#initial_trace = sum((np.real(trace(test_X.dot(sigma)))-test_y)**2)

def loss(sigma_real, sigma_imag):
    sigma_real, sigma_imag = cholesky(sigma_real, sigma_imag)
    res = 0
    idx = np.random.choice(np.arange(train_X.shape[0]), batch_size)
    for E_m,y_m in zip(train_X[idx], train_y[idx].astype('float64')):
        E_real, E_imag = [tn.Tensor(x) for x in [np.real(E_m), np.imag(E_m)]]
        res += ((E_real.dot(sigma_real)+E_imag.dot(sigma_imag)-y_m)**2)
#     return res/(initial_trace*train_X.shape[0])
    return res
    

def eval_loss(sigma_real, sigma_imag): # any score function can be used here
    sigma_real_, sigma_imag_ = cholesky(sigma_real, sigma_imag)
    sigma = sigma_real_.torch().detach().cpu().numpy() + 1j*sigma_imag_.torch().detach().cpu().numpy()
    return -lib.fidelity(sigma, rho)

#print('Trace before: %f'%initial_trace)

lib.tn_optimize([sigma_real, sigma_imag], loss, eval_loss, tol=0, patience=1000,print_freq=10,lr=lr)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
iter: 0 |  loss: 33.11676025390625 | eval_loss: -0.5860968114111752 | total time: 0.16710543632507324
iter: 10 |  loss: 32.244232177734375 | eval_loss: -0.6025110824562516 | total time: 1.5547106266021729
iter: 20 |  loss: 28.429380416870117 | eval_loss: -0.6025762969815024 | total time: 2.936919689178467
iter: 30 |  loss: 27.644153594970703 | eval_loss: -0.6026209020350124 | total time: 4.3172266483306885
iter: 40 |  loss: 28.58127212524414 | eval_loss: -0.6026386650438132 | total time: 5.701262474060059
iter: 50 |  loss: 26.781190872192383 | eval_loss: -0.6027080739468862 | total time: 7.084668874740601
iter: 60 |  loss: 25.110837936401367 | eval_loss: -0.6027198753265989 | total time: 8.497791528701782
iter: 70 |  loss: 31.821475982666016 | eval_loss: -0.6029248329029296 | total time: 9.90051817893982
iter: 80 |  loss: 28.520763397216797 | eval_loss: -0.602796369289432 | total time: 

iter: 800 |  loss: 26.378082275390625 | eval_loss: -0.5306861481709461 | total time: 111.89225745201111
iter: 810 |  loss: 25.092790603637695 | eval_loss: -0.530704846524837 | total time: 113.3053936958313
iter: 820 |  loss: 25.703317642211914 | eval_loss: -0.5310325215157987 | total time: 114.70702075958252
iter: 830 |  loss: 26.389122009277344 | eval_loss: -0.5315268546626659 | total time: 116.08695268630981
iter: 840 |  loss: 24.25170135498047 | eval_loss: -0.5317761634913366 | total time: 117.51441144943237
iter: 850 |  loss: 22.186155319213867 | eval_loss: -0.5319385589335645 | total time: 118.95687198638916
iter: 860 |  loss: 23.096206665039062 | eval_loss: -0.5322230917818243 | total time: 120.34032773971558
iter: 870 |  loss: 22.78999137878418 | eval_loss: -0.5323123204841729 | total time: 121.73070907592773
iter: 880 |  loss: 23.137069702148438 | eval_loss: -0.5327695109840619 | total time: 123.13912606239319
iter: 890 |  loss: 27.09807777404785 | eval_loss: -0.532968722686585

In [4]:
sigma_real_, sigma_imag_ = cholesky(sigma_real, sigma_imag)
sigma = sigma_real_.torch().detach().cpu().numpy() + 1j*sigma_imag_.torch().detach().cpu().numpy()
sigma /= sum([sigma[i,i] for i in range(int(2**n_qubits))])
lib.fidelity(sigma, rho)

0.5396036625798617